In [1]:
import numpy as np
import scipy 

In [2]:
def f(x):
    return 0.5 * np.cos(2*x) * np.exp(2*x/5) + 2.4 * np.sin(1.5 *x) * np. exp(-6* x) + 6* x

In [3]:
def omg(x , n = 3, a = 1.1, b = 2.5):
    res = 1
    for i in range(n):
        res*=(x - a+(b-a)*i/(n-1))
    return res

In [4]:
def p(x, a = 1.1, b = 2.5):
    return (x-a)**(-2/5)

In [5]:
def mu(k, a = 0, b = 1.4):
    return b**(k+3/5)/(k+3/5) - a**(k+3/5)/(k+3/5)

In [6]:
def sIQF_NewtonKots(a = 1.1, z1 = 1.1, z2 = 2.5, n = 3):
    m = np.zeros_like(n, shape = n, dtype = float)
    for i in range(n):
        m[i] = mu(i, z1 - a, z2 - a)
    X = np.zeros_like(n^2, shape = (n,n), dtype = float)    
    for i in range(n):
        for j in range(n):
            X[i][j] = ((z1-a) + (z2-z1)*j/(n-1))**i
    A = np.linalg.solve(X, m)
    S = 0
    for i in range(n):
        S+=A[i]*f(X[1][i] + a)
    return S

In [7]:
def cIQF_NewtonKots(a = 1.1, b = 2.5, n = 3, k = 6):
    res = 0
    for i in range(k):
        add = sIQF_NewtonKots(a,a+(b-a)*i/k, a+(b-a)*(i+1)/k, n) 
        res+=add
    return res

In [8]:
def Etkin(Sh, L = 2):
    return -np.log(abs((Sh[-1]-Sh[-2])/(Sh[-2]-Sh[-3])))/np.log(L)

In [9]:
def Richardson(Sh, h0 = 0.9, L = 2):
    m = Etkin(Sh)
    n = len(Sh)
    A = np.zeros_like(n**2, shape = (n,n), dtype = float)
    for i in range(n):
        for j in range(n-1):
            A[i][j] = (h0/L**i)**(m+j)
        A[i][n-1] = 1
    x = np.linalg.solve(A, Sh)
    return x[-1]

In [10]:
def Runge(Sh, L = 2):
    m = Etkin(Sh)
    Rh = [(Sh[-1] - Sh[-2])/(1-L**(-m)), (Sh[-1] - Sh[-2])/(-1+L**(m))]
    return Rh

In [11]:
acc = scipy.integrate.quad(lambda x: f(x)*p(x),1.1,2.5)[0]
print(acc)

18.60294785732012


In [12]:
#Малая ИКФ типа Ньютона-Котса
sIQF_NewtonKots()

18.55804137623109

## Составная ИКФ типа Ньютона-Котса

In [13]:

a = 1.1
b = 2.5
eps = 1e-6
Sh = [cIQF_NewtonKots(k=2), cIQF_NewtonKots(k=4), cIQF_NewtonKots(k=8)]
Rh = [1.,1.,1.]
c = 3;
while(abs(Rh[-1])> eps):
    c+=1
    Sh.append(cIQF_NewtonKots(k=2**c))
    m = Etkin(Sh)
    Rh.append(Richardson(Sh, b - a)-Sh[-1])
    print("Шаг:", c, "Вычисленное значение:", Sh[-1], "Ошибка:", Rh[-1],"Степень по Эйткину:" , m)
combNewton =  Sh[-1]

Шаг: 4 Вычисленное значение: 18.602944794025312 Ошибка: 3.1069164307950814e-06 Степень по Эйткину: 3.5306567120217025
Шаг: 5 Вычисленное значение: 18.602947599346283 Ошибка: 2.597568702356057e-07 Степень по Эйткину: 3.556578606022004


## Составная ИКФ типа Ньютона-Котса с выбором оптимального шага

In [14]:

a = 1.1
b = 2.5
eps = 1e-6
Sh = [cIQF_NewtonKots(k=2), cIQF_NewtonKots(k=4), cIQF_NewtonKots(k=8)]

Rh = [1.]
m = Etkin(Sh)
Rh+=Runge(Sh, (b-a)/2)
Kopt = np.ceil((b-a)/(np.math.pow(eps/abs(Rh[-1]),1/m)*((b-a)*2**(-3))*0.95))
print('Kopt =', Kopt)
Sh = [cIQF_NewtonKots(k = Kopt), cIQF_NewtonKots(k = Kopt*2)]
Kopt*=4
Rh=[1.]
c = 2
while(abs(Rh[-1])> eps):
    Sh.append(cIQF_NewtonKots(k=Kopt))
    m = Etkin(Sh)
    Rh.append(Richardson(Sh, b - a)-Sh[-1])
    Kopt*=2
    c+=1
    print("Шаг:", c, "Вычисленное значение:", Sh[-1], "Ошибка:", Rh[-1],"Степень по Эйткину:" , m)
combNewton_withHopt =  Sh[-1]

Kopt = 52.0


TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [ ]:
def sIQF_Gaus(a = 1.1, z1 = 1.1, z2 = 2.5, n = 3):
    m = np.zeros_like(2*n, shape = 2*n, dtype = float)
    for i in range(2*n):
        m[i] = mu(i, z1 - a, z2 - a)
   
    X = np.zeros_like(n^2, shape = (n,n), dtype = float)
    for i in range(n):
        for j in range(n):
            X[i][j] = m[i+j]
            
    b = -m[n:]
    A = np.linalg.solve(X, b)
    A = np.append(A, 1.)
    A=A[::-1]
    x = np.roots(A)
    for i in range(n):
        X[i][:] = x**i
    m=m[0:n]
    A = np.linalg.solve(X, m)
    S = 0
    for i in range(n):
        S += A[i]*f(X[1][i] + a)
    return S

In [ ]:
def cIQF_Gaus(a = 1.1, b = 2.5, n = 3, k = 6):
    res = 0
    for i in range(k):
        add = sIQF_Gaus(a,a+(b-a)*i/k, a+(b-a)*(i+1)/k, n) 
        res+=add
    return res

## Простая ИКФ типа Гаусса

In [ ]:

sIQF_Gaus()

18.602517777238724

## Составная ИКФ типа Гаусса

In [ ]:

a = 1.1
b = 2.5
eps = 1e-6
Sh = [cIQF_Gaus(k=2), cIQF_Gaus(k=4), cIQF_Gaus(k=8)]
Rh = [1.,1.,1.]
c = 3;
while(abs(Rh[-1])> eps):
    c+=1
    Sh.append(cIQF_Gaus(k=2**c))
    m = Etkin(Sh)
    Rh.append(Richardson(Sh, b - a)-Sh[-1])
    print("Шаг:", c, "Вычисленное значение:", Sh[-1], "Ошибка:", Rh[-1],"Степень по Эйткину:" , m)
combNewton =  Sh[-1]

Шаг: 4 Вычисленное значение: 18.602947857299732 Ошибка: 1.858069254012662e-11 Степень по Эйткину: 6.035456874711196


## Составная ИКФ типа Гаусса с выбором оптимального шага

In [ ]:

a = 1.1
b = 2.5
eps = 1e-6
Sh = [cIQF_Gaus(k=2), cIQF_Gaus(k=4), cIQF_Gaus(k=8)]

Rh = [1.]
m = Etkin(Sh)
Rh+=Runge(Sh, (b-a)/2)
Kopt = np.ceil((b-a)/(np.pow(eps/abs(Rh[-1]),1/m)*((b-a)*2**(-3))*0.95))
print('Kopt =', Kopt)
Sh = [cIQF_Gaus(k = Kopt), cIQF_Gaus(k = Kopt*2)]
Kopt*=4
Rh=[1.]
c = 2
while(abs(Rh[-1])> eps):
    Sh.append(cIQF_Gaus(k=Kopt))
    m = Etkin(Sh)
    Rh.append(Richardson(Sh, b - a)-Sh[-1])
    Kopt*=2
    c+=1
    print("Шаг:", c, "Вычисленное значение:", Sh[-1], "Ошибка:", Rh[-1],"Степень по Эйткину:" , m)
combNewton_withHopt =  Sh[-1]

Kopt = 6
Шаг: 3 Вычисленное значение: 18.602947857316842 Ошибка: 1.623590151211829e-12 Степень по Эйткину: 6.024472760077107
